In [3]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.plotting.plotting_main import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )

# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../data/prices.hd5')
rsi_ind = from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    cand_ohlc="close",
)

In [4]:
price_data

symbol                BTCUSDT                               EOSUSD            \
candle_info              open      high       low     close   open high  low   
open_time                                                                      
2020-10-01 00:00:00 10,759.00 10,817.50 10,759.00 10,790.00   2.58 2.58 2.58   
2020-10-01 00:30:00 10,790.00 10,790.50 10,775.00 10,777.00   2.58 2.58 2.58   
2020-10-01 01:00:00 10,777.00 10,793.00 10,777.00 10,793.00   2.58 2.59 2.58   
2020-10-01 01:30:00 10,793.00 10,836.00 10,792.50 10,822.00   2.59 2.59 2.59   
2020-10-01 02:00:00 10,822.00 10,841.50 10,808.00 10,808.50   2.59 2.60 2.59   
...                       ...       ...       ...       ...    ...  ...  ...   
2021-04-14 21:30:00 62,703.50 62,812.50 62,615.50 62,795.00   7.58 7.62 7.54   
2021-04-14 22:00:00 62,795.00 62,825.00 62,591.00 62,680.00   7.61 7.73 7.55   
2021-04-14 22:30:00 62,680.00 62,993.00 62,680.00 62,849.50   7.65 7.77 7.65   
2021-04-14 23:00:00 62,849.50 62,999.00 62,750.00 62,999.00   7.71 7.79 7.67   
2021-04-14 23:30:00 62,999.00 62,999.00 62,606.00 62,921.00   7.78 7.78 7.63   

symbol                      ETHUSD                            LTCUSDT         \
candle_info         close     open     high      low    close    open   high   
open_time                                                                      
2020-10-01 00:00:00  2.58   359.85   361.45   358.85   358.95     NaN    NaN   
2020-10-01 00:30:00  2.58   358.95   359.75   358.95   359.25     NaN    NaN   
2020-10-01 01:00:00  2.59   359.25   359.80   358.80   359.75     NaN    NaN   
2020-10-01 01:30:00  2.59   359.75   361.40   359.75   361.20     NaN    NaN   
2020-10-01 02:00:00  2.59   361.20   364.05   361.20   362.95     NaN    NaN   
...                   ...      ...      ...      ...      ...     ...    ...   
2021-04-14 21:30:00  7.61 2,406.20 2,440.00 2,397.20 2,434.05  269.26 281.37   
2021-04-14 22:00:00  7.65 2,434.05 2,445.00 2,418.65 2,427.00  280.82 282.02   
2021-04-14 22:30:00  7.71 2,427.00 2,438.60 2,423.50 2,431.30  278.49 281.31   
2021-04-14 23:00:00  7.78 2,431.30 2,450.00 2,431.30 2,445.95  277.50 280.17   
2021-04-14 23:30:00  7.68 2,445.95 2,445.95 2,417.40 2,438.20  279.05 280.95   

symbol                            XRPUSD                  
candle_info            low  close   open high  low close  
open_time                                                 
2020-10-01 00:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 00:30:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 01:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 01:30:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 02:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
...                    ...    ...    ...  ...  ...   ...  
2021-04-14 21:30:00 269.11 280.82   1.78 1.82 1.77  1.82  
2021-04-14 22:00:00 278.24 278.49   1.82 1.82 1.77  1.78  
2021-04-14 22:30:00 276.64 277.50   1.78 1.82 1.78  1.80  
2021-04-14 23:00:00 277.46 279.05   1.80 1.84 1.79  1.83  
2021-04-14 23:30:00 277.95 279.41   1.83 1.85 1.82  1.84  

[9408 rows x 20 columns]

In [5]:
rsi_ind

symbol              BTCUSDT       EOSUSD       ETHUSD       LTCUSDT        \
RSI_timeperiod           15    30     15    30     15    30      15    30   
open_time                                                                   
2020-10-01 00:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 00:30:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 01:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 01:30:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 02:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
...                     ...   ...    ...   ...    ...   ...     ...   ...   
2021-04-14 21:30:00   45.28 47.78  53.37 52.99  66.12 62.18   63.93 58.31   
2021-04-14 22:00:00   43.88 46.99  54.99 53.83  64.17 61.16   60.58 56.59   
2021-04-14 22:30:00   46.50 48.29  56.95 54.84  64.85 61.55   59.17 55.86   
2021-04-14 23:00:00   48.76 49.41  59.35 56.10  67.11 62.89   60.71 56.76   
2021-04-14 23:30:00   47.64 48.84  54.74 53.91  64.75 61.72   61.07 56.97   

symbol              XRPUSD        
RSI_timeperiod          15    30  
open_time                         
2020-10-01 00:00:00    NaN   NaN  
2020-10-01 00:30:00    NaN   NaN  
2020-10-01 01:00:00    NaN   NaN  
2020-10-01 01:30:00    NaN   NaN  
2020-10-01 02:00:00    NaN   NaN  
...                    ...   ...  
2021-04-14 21:30:00  56.21 53.93  
2021-04-14 22:00:00  52.11 51.85  
2021-04-14 22:30:00  53.82 52.75  
2021-04-14 23:00:00  56.84 54.34  
2021-04-14 23:30:00  57.71 54.81  

[9408 rows x 10 columns]

In [6]:
entries

symbol              BTCUSDT                      EOSUSD                       \
RSI_timeperiod           15            30            15            30          
RSI_is_below             40     50     40     50     40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00   False   True  False   True  False  False  False  False   
2021-04-14 22:00:00   False   True  False   True  False  False  False  False   
2021-04-14 22:30:00   False   True  False   True  False  False  False  False   
2021-04-14 23:00:00   False   True  False   True  False  False  False  False   
2021-04-14 23:30:00   False   True  False   True  False  False  False  False   

symbol              ETHUSD                      LTCUSDT                       \
RSI_timeperiod          15            30             15            30          
RSI_is_below            40     50     40     50      40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00  False  False  False  False   False  False  False  False   
2020-10-01 00:30:00  False  False  False  False   False  False  False  False   
2020-10-01 01:00:00  False  False  False  False   False  False  False  False   
2020-10-01 01:30:00  False  False  False  False   False  False  False  False   
2020-10-01 02:00:00  False  False  False  False   False  False  False  False   
...                    ...    ...    ...    ...     ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  False  False   False  False  False  False   
2021-04-14 22:00:00  False  False  False  False   False  False  False  False   
2021-04-14 22:30:00  False  False  False  False   False  False  False  False   
2021-04-14 23:00:00  False  False  False  False   False  False  False  False   
2021-04-14 23:30:00  False  False  False  False   False  False  False  False   

symbol              XRPUSD                       
RSI_timeperiod          15            30         
RSI_is_below            40     50     40     50  
open_time                                        
2020-10-01 00:00:00  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  
...                    ...    ...    ...    ...  
2021-04-14 21:30:00  False  False  False  False  
2021-04-14 22:00:00  False  False  False  False  
2021-04-14 22:30:00  False  False  False  False  
2021-04-14 23:00:00  False  False  False  False  
2021-04-14 23:30:00  False  False  False  False  

[9408 rows x 20 columns]

In [7]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=price_data,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(6, 8.1, 1),
    risk_rewards=np.arange(2, 5.1, 0.5),
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(6, 9.1, 0.5),
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=np.arange(6, 9.1, 0.5),
    tsl_when_pct_from_avg_entry=np.arange(2, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 3,087
Total order settings to test: 15,435
Total candles per symbol: 9,408
Total candles to test: 580,849,920

Total combinations to test: 61,740


In [13]:
strat_results_df

,symbol,entries_col,or_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
10608,ETHUSD,11,2449,52.00,358.35,46.15,0.90,"3,583.46","4,583.46"
9895,ETHUSD,11,1420,52.00,289.17,46.15,0.90,"2,891.71","3,891.71"
9894,ETHUSD,11,1419,55.00,274.51,43.64,0.90,"2,745.06","3,745.06"
9460,ETHUSD,11,412,51.00,224.00,47.06,0.90,"2,239.96","3,239.96"
11189,LTCUSDT,13,120,73.00,193.13,43.84,0.90,"1,931.28","2,931.28"
...,...,...,...,...,...,...,...,...,...
954,BTCUSDT,0,1428,42.00,153.38,50.00,0.60,"1,533.77","2,533.77"
3099,BTCUSDT,1,1933,49.00,241.09,48.98,0.60,"2,410.92","3,410.92"
3730,BTCUSDT,1,2778,37.00,162.82,54.05,0.60,"1,628.20","2,628.20"
3820,BTCUSDT,1,2925,37.00,170.78,54.05,0.60,"1,707.85","2,707.85"


In [15]:
settings_results_df.iloc[:,10608]

symbol                            ETHUSD
entries_col                           11
max_equity_risk_pct                 8.00
risk_rewards                        3.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       8.00
tsl_trail_by_pct                    8.00
tsl_when_pct_from_avg_entry         3.00
Name: 10608, dtype: object

In [16]:
order_records = simulate_up_to_6_nb(
    prices=price_data['ETHUSD'].values,
    entries=entries.iloc[:,[11]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=8,
    risk_rewards=3,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=CandleBody.low,
    tsl_pcts_init=8,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [19]:
entries.iloc[:,[11]]

symbol,ETHUSD
RSI_timeperiod,30
RSI_is_below,50
open_time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-04-14 21:30:00,False


In [18]:
rsi_ind[[('ETHUSD', 30)]]

symbol,ETHUSD
RSI_timeperiod,30
open_time,
2020-10-01 00:00:00,NaN
2020-10-01 00:30:00,NaN
2020-10-01 01:00:00,NaN
2020-10-01 01:30:00,NaN
2020-10-01 02:00:00,NaN
...,...
2021-04-14 21:30:00,62.18
2021-04-14 22:00:00,61.16


In [20]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('ETHUSD', 30)]],
        "entries1": entries.iloc[:,[11]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.ETHUSD,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
